In [75]:
%load_ext autoreload
%autoreload 2

## API endpoints:


I. Endpoints for fetching rules

- Give me all the rules
- Give me rules for the datatypes X,Y,Z
- Give me rules applicable to the datatypes X,Y,Z given input filters

They return:
- a list of rules with their meta-data: id/name/desc (premise/query ?)

---------------------

II. Endpoinds for applying rules

- Apply rule A given input filters
- Apply rules A,B,C given input filters (?)

They return:
- a list of result resource IDs or full payloads (ids passed to ES views by atals?)


ISSUES TO RESOLVE:

- Convention for the filters: how atlas knows which params need to be specified for different rules
- Permission:
    - fetching rules: service account that can always access the project with rules.
    - applying rules: we need to get USER token from atlas and use it when applying the rules. handle access denied in some sessions.
- Service sessions for storing forge sessions for different users.
- Pagination of results.

In [114]:
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule

In [77]:
ENDPOINT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"

In [78]:
TOKEN = getpass.getpass()

········


In [79]:
RULES_ORG = "dke"
RULES_PROJECT = "inference-test"
RULES_VIEW = "https://bbp.epfl.ch/neurosciencegraph/data/rule-view"

In [145]:
forge = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{RULES_ORG}/{RULES_PROJECT}")

# Fetch rules

1. Fetch all the rules

In [162]:
rules = fetch_rules(forge, RULES_VIEW)
for r in rules:
#     print(r)
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  Trace generalization for cortical PCs
Desc:  Collect traces belonging to PCs from other cortical regions
Target resource type:  Trace
Input parameters: 
	 brainRegion
		 {'@type': 'sparql_uri', 'description': 'URI of the query brain region', 'name': 'brainRegion'}
	 MType
		 {'@type': 'sparql_uri', 'description': 'URI of the query MType', 'name': 'MType'}

Name:  MType similarity-based trace generalization for cortical PCs
Desc:  Collect traces belonging to the closest (PC) MType from other cortical regions
Target resource type:  Trace
Input parameters: 
	 brainRegion
		 {'@type': 'sparql_uri', 'description': 'URI of the query brain region', 'name': 'brainRegion'}
	 MType
		 {'@type': 'str', 'description': 'URI of the query MType', 'name': 'MType'}

Name:  Combined (PC) MType similarity
Desc:  Get the closest (PC) MType according to a combined similarity
Target resource type:  Class
Input parameters: 
	 MType
		 {'@type': 'str', 'description': 'URI of the query MType', 'name': '

2. Fetch rules for a particular data type

In [161]:
trace_rules = fetch_rules(forge, RULES_VIEW, resource_types=["Trace"])
for r in trace_rules:
#     print(r)
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  Trace generalization for cortical PCs
Desc:  Collect traces belonging to PCs from other cortical regions
Target resource type:  Trace
Input parameters: 
	 brainRegion
		 {'@type': 'sparql_uri', 'description': 'URI of the query brain region', 'name': 'brainRegion'}
	 MType
		 {'@type': 'sparql_uri', 'description': 'URI of the query MType', 'name': 'MType'}

Name:  MType similarity-based trace generalization for cortical PCs
Desc:  Collect traces belonging to the closest (PC) MType from other cortical regions
Target resource type:  Trace
Input parameters: 
	 brainRegion
		 {'@type': 'sparql_uri', 'description': 'URI of the query brain region', 'name': 'brainRegion'}
	 MType
		 {'@type': 'str', 'description': 'URI of the query MType', 'name': 'MType'}



# Check premises of rules (find rules applicable to input filters)

Altas sends to the service user selection from filters:

- Atlas
- Brain Region
- Cell Type
- Data Type

and advanced queries (?)

Example input values:

In [171]:
input_filters_a = {
    "brainRegion": "http://api.brain-map.org/api/v2/data/Structure/31",
    "MType": "http://uri.interlex.org/base/ilx_0381377",
    "species": "http://purl.obolibrary.org/obo/NCBITaxon_10088",
}

In [172]:
input_filters_b = {
    "brainRegion": "http://api.brain-map.org/api/v2/data/Structure/345",
    "MType": "http://uri.interlex.org/base/ilx_0381377",
    "species": "http://purl.obolibrary.org/obo/NCBITaxon_10088",
}

If premise is satified given the input filters, the rule is applicable.

In [176]:
for rule in rules:
    print(rule["name"])
    print("\t-> Input filters A:")
    print("\t\tPremise satisfied: ", check_premises(rule, input_filters_a, TOKEN))
    print("\t-> Input filters B:")
    print("\t\tPremise satisfied: ", check_premises(rule, input_filters_b, TOKEN))
    print()

Trace generalization for cortical PCs
	-> Input filters A:
		Premise satisfied:  True
	-> Input filters B:
		Premise satisfied:  False

MType similarity-based trace generalization for cortical PCs
	-> Input filters A:
		Premise satisfied:  True
	-> Input filters B:
		Premise satisfied:  False

Combined (PC) MType similarity
	-> Input filters A:
		Premise satisfied:  True
	-> Input filters B:
		Premise satisfied:  True



# Excecute inference

Rule application returns a collection of resources to display in the atlas

In [191]:
for rule in rules:
    print(rule["name"])
    print("\t Results:")
    results = apply_rule(rule, input_filters, TOKEN)
    for r in results:
        print("\t\t", r)
    print()

Trace generalization for cortical PCs
	 Results:
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/2b108ccf-4ad2-4495-8490-17b2fde656fa'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/4fba4ab3-189c-4ff4-96bf-6904c1b8912b'}

MType similarity-based trace generalization for cortical PCs
	 Results:
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'}

Combined (PC) MType similarity
	 Results:
		 {'id': 'http://uri.interlex.org/base/ilx_0381371'}

